In [1]:
import numpy as np
import string
from PIL import Image, ImageFont, ImageDraw

import numpy as np
import os
import string
from skimage.io import imread
from TFANN import CNNR

/Users/ammar/Desktop/online/.env/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def MakeImg(t, f, fn, s = (100, 100), o = (16, 8)):
    '''
    Generate an image of text
    t:      The text to display in the image
    f:      The font to use
    fn:     The file name
    s:      The image size
    o:      The offest of the text in the image
    '''
    img = Image.new('RGB', s, "black")
    draw = ImageDraw.Draw(img)
    draw.text((0,0), t, (255, 255, 255), font = f)
    img.save(fn)
 

In [3]:
#The possible characters to use
CS = list("ہدونچسکتاثر")
RTS = list(np.random.randint(10, 64, size = 16284)) + [64]

#The random strings
S = [''.join(np.random.choice(CS, i)) for i in RTS]

#Get the font
font = ImageFont.truetype("urdu1.ttf", 16)

#The largest size needed
MS = max(font.getsize(Si) for Si in S)

#Computed offset
OFS = ((640 - MS[0]) // 2, (32 - MS[1]) // 2)

#Image size
MS = (640, 32)
Y = []
for i, Si in enumerate(S):
    MakeImg(Si, font, str(i) + '.png', MS, OFS)
    Y.append(str(i) + '.png,' + Si)

#Write CSV file
with open('Train.csv', 'w') as F:
    F.write('\n'.join(Y))

In [4]:
#The possible chars
CS = ['\x00'] + CS
#Map from char to index
CM = dict(zip(CS, range(len(CS))))
#Number of possible chars
NC = len(CM)

In [5]:
if __name__ == "__main__":
    FP = '/Users/ammar/Desktop/online22222'
    TFP = os.path.join(FP, 'Train.csv')
    A, Y, T = [], [], []
    with open(TFP) as F:
        for Li in F:
            FNi, Yi = Li.strip().split(',')         #filename,string
            Yi = (Yi + ('\x00' * (64 - len(Yi))))   #Pad the string with null chars
            R = [[0 if i != CM[j] else 1 for i in range(NC)] for j in Yi]
            Y.append(R)
            A.append(imread(os.path.join(FP, FNi)))
            T.append(Yi)
    A = np.stack(A)     #Input images
    Y = np.stack(Y)     #Target strings in 1-hot encoding
    #Architecture of the neural network

In [6]:
#Architecture of the neural network
ws = [('C', [4, 4,  3, NC // 2], [1, 2, 2, 1]), ('C', [4, 4, NC // 2, NC], [1, 2, 1, 1]), ('C', [8, 5, NC, NC], [1, 8, 5, 1]), ('R', [-1, 64, NC])]

In [7]:
#Create the neural network in TensorFlow
cnnc = CNNR(A[0].shape, ws, batchSize=32, learnRate=1e-3, loss='smce', maxIter= 32, reg= 6e-4, tol= 1e-2, verbose= True)


In [8]:
#Fit the network
cnnc.fit(A, Y)
#The predictions as sequences of character indices
YH = np.zeros((Y.shape[0], Y.shape[1]), dtype = np.int)
for i in np.array_split(np.arange(A.shape[0]), 32): 
    YH[i] = np.argmax(cnnc.predict(A[i]), axis = 2)
#Convert from sequence of char indices to strings
PS = [''.join(CS[j] for j in YHi) for YHi in YH]
for PSi, Ti in zip(PS, T):
    print(Ti + '\t->\t' + PSi)

Iter     1	      2.48276454 (Batch Size:    32)
Iter     2	      2.47097562 (Batch Size:    32)
Iter     3	      2.46555817 (Batch Size:    32)
Iter     4	      2.46209577 (Batch Size:    32)
Iter     5	      2.46148225 (Batch Size:    32)
Iter     6	      2.46099367 (Batch Size:    32)
Iter     7	      2.45909098 (Batch Size:    32)
Iter     8	      2.45637863 (Batch Size:    32)
Iter     9	      2.45616087 (Batch Size:    32)
Iter    10	      2.42789533 (Batch Size:    32)
Iter    11	      1.72383094 (Batch Size:    32)
Iter    12	      1.71703597 (Batch Size:    32)
Iter    13	      1.71946635 (Batch Size:    32)
Iter    14	      1.72936787 (Batch Size:    32)
Iter    15	      1.71176981 (Batch Size:    32)
Iter    16	      1.71888335 (Batch Size:    32)
Iter    17	      1.71611481 (Batch Size:    32)
Iter    18	      1.70985932 (Batch Size:    32)
Iter    19	      1.70626786 (Batch Size:    32)
Iter    20	      1.70314524 (Batch Size:    32)
Iter    21	      1.70296990 (Batch Size:

چاچہاثوکنوراچنرثثسرتہثساہکتداہددرواننککا                        	->	چاہثاندچنثثسوتثراکتدردورکد                                      
ثودثوووہوہکددوکثتاسوترترسدناوکددہہکاثننثاتثرککچہرہرنا           	->	ثودووہہکسکتاسورترسدوکدہکثرثاثاککہہر                             
ثاہتثکتہاوک                                                     	->	ثاثثاتہ                                                         
ااسساچندوسہہوچاثچاتسنث                                          	->	اسرانثسہچاچتراث                                                 
کثتادسہچاثکثادثکثثروس                                           	->	کثااسہکککاثکثثرس                                                
نتکدادونسرثہستاچوہدچچددچاچسسچوثکننہروہاوتوچکثرر                 	->	نتکدسنرثہستچہچچورہسچثکننرہاوچثر                                 
سچثکنکررہثنداچچواثنرنوکرتثستثہچچکرچونہ                          	->	سچثکنکرثسنچوثرنکرثتستثچچرچ                                      
دسنوچسااندرہثدثکثاچسرککچندثنندرہدرسوہثسچثاوہرددچسسکتساکچسچنک    	->	د

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

